In [1]:
import create_population
import preprocessing
from feature_engineering import FeatureCreator
from feature_engineering_prediction import PredictionFeatureCreator

import prediction

import pandas as pd


import html5lib

%load_ext autoreload

In [2]:
%autoreload

In [3]:
population = create_population.create(from_="2019-01-01", to_="2023-12-31")


In [18]:
population 

,race_id,date,horse_id
0,201906010105,2019-01-05,2016104656
1,201906010105,2019-01-05,2016103323
2,201906010105,2019-01-05,2016105106
3,201906010105,2019-01-05,2016102787
4,201906010105,2019-01-05,2016102128
...,...,...,...
85562,202309050912,2023-12-28,2017104873
85563,202309050912,2023-12-28,2019100653
85564,202309050912,2023-12-28,2017106240
85565,202309050912,2023-12-28,2018103205


In [26]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.reset_option('display.precision')
pd.reset_option('display.float_format')

In [5]:
#futre_修正、馬場状態
# レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()
# 馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()
# レース情報テーブルの前処理
race_info_preprocessed = preprocessing.process_race_info()

In [6]:
from pathlib import Path

from tqdm.notebook import tqdm
import json
import re
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO 

DATA_DIR = Path("..", "data")
POPULATION_DIR = DATA_DIR / "00_population"
INPUT_DIR = DATA_DIR / "01_preprocessed"
OUTPUT_DIR = DATA_DIR / "02_features"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

population_dir = POPULATION_DIR
poplation_filename = "population.csv"
input_dir = INPUT_DIR
results_filename = "results.csv"
race_info_filename = "race_info.csv"
horse_results_filename = "horse_results.csv"
jockey_leading_filename = "jockey_leading.csv"
trainer_leading_filename = "trainer_leading.csv"
peds_filename = "peds.csv"
sire_leading_filename = "sire_leading.csv"
output_dir = OUTPUT_DIR
output_filename = "features.csv"
population = pd.read_csv(population_dir / poplation_filename, sep="\t")
results = pd.read_csv(input_dir / results_filename, sep="\t")
race_info = pd.read_csv(input_dir / race_info_filename, sep="\t")
horse_results = pd.read_csv(input_dir / horse_results_filename, sep="\t")
jockey_leading = pd.read_csv(input_dir / jockey_leading_filename, sep="\t")
trainer_leading = pd.read_csv(
    input_dir / trainer_leading_filename, sep="\t"
)
peds = pd.read_csv(input_dir / peds_filename, sep="\t")
sire_leading = pd.read_csv(input_dir / sire_leading_filename, sep="\t")
output_dir = output_dir
output_filename = output_filename
agg_horse_per_group_cols_dfs = {}

In [7]:

"""
horse_resultsをレース結果テーブルの日付よりも過去に絞り、集計元のログを作成。
"""
df = (
    results[["race_id", "horse_id","mean_age_kirisute"]]
    .merge(race_info[["race_id", "race_type", "season","race_class"]], on="race_id")
)
race_info = race_info.copy()


"""

dfに新たな列、race_gradeを作成して欲しい
作成ルールは以下の通りである
'age_season'の条件に引っかかった場合、それを優先すること
次点で"race_class"の条件にかかっても、'age_season'がある方を優先して変換すること


"race_class"列が0は55

"race_class"列が1は60

"race_class"列が2は70
2歳それ以外は68（20<='age_season'<30かつ、2<="race_class"列<5の行）
2歳G2,G3,OPは73（20<='age_season'<30かつ、5<="race_class"列<8の行）

"race_class"列が3は79
2歳G1は79（20<='age_season'<30かつ、8<="race_class"の行）
3歳春OPは80（30<='age_season'<33かつ、4<="race_class"列<6の行）
3歳春G2.G3は81（30<='age_season'<33かつ、6<="race_class"列<8の行）

"race_class"列が4は85
3歳春G1は86（30<='age_season'<33かつ、8<="race_class"の行）
3歳秋G2,G3は86（33<='age_season'<40かつ、5<="race_class"列<8の行）

"race_class"列が5は89
3歳秋G1は91（33<='age_season'<40かつ、8<="race_class"の行）

"race_class"列が6は92

"race_class"列が7は94

"race_class"列が8は98




これらを小さく（1/10 - 5）した列

G1 8	100
G2 7	95
G3 6	92
オープン5	89
1600万4	86
２勝クラス3	80
１勝クラス2	70
未勝利1	60
新馬0	55


クラス	芝	ダート
未勝利	６５（-１５）	６０（-２０）
500万下
Ｇ１を除く２歳ＯＰ	７５（-５）	７２（-８）
1000万下
２歳Ｇ１
Ｇ１を除く３歳春ＯＰ	８３（３）	８３（３）
1600万下
３歳春Ｇ１
３歳秋重賞	８８（８）	９０（１０）
ＯＰ（ただしダート重賞を除く）
３歳秋Ｇ１	９３（１３）	９５（１５）
ダート重賞（３歳を除く）	－	１００（２０）
古馬Ｇ１	９８（１８）	１０５（２５）
"""
# "mean_age_kirisute"と"season"を文字列に変換して結合し、int型に変換して新しい列 "age_season" を作成
df['age_season'] = (df['mean_age_kirisute'].astype(str) + df['season'].astype(str)).astype(int)

# race_gradeの作成
def calculate_race_grade(row):
    age_season = row['age_season']
    race_class = row['race_class']

    # 'age_season' に基づく条件を優先してチェック
    if 20 <= age_season < 30:
        if 2 <= race_class < 5:
            return 70
        elif 5 <= race_class < 8:
            return 70
        elif 8 <= race_class:
            return 79
    elif 30 <= age_season < 33:
        if 4 <= race_class < 6:
            return 79
        elif 6 <= race_class < 8:
            return 79
        elif 8 <= race_class:
            return 85
    elif 33 <= age_season < 40:
        if 5 <= race_class < 8:
            return 85
        elif 8 <= race_class:
            return 91
    
    if race_class == 0:
        return 55
    elif race_class == 1:
        return 60
    elif race_class == 2:
        return 70
    elif race_class == 3:
        return 79
    elif race_class == 4:
        return 85
    elif race_class == 5:
        return 89
    elif race_class == 6:
        return 91
    elif race_class == 7:
        return 94
    elif race_class == 8:
        return 98
    else:
        return np.nan  

# race_grade列を作成
df['race_grade'] = df.apply(calculate_race_grade, axis=1)

#race_grade_scaledの作成
df['race_grade_scaled'] = df['race_grade'] / 10 - 5

race_info[['age_season', 'race_grade', 'race_grade_scaled']] = df[['age_season', 'race_grade', 'race_grade_scaled']]
race_info.to_csv(input_dir/race_info_filename, sep="\t", index=False)


In [8]:
n_races = [1, 3, 5, 10]

In [112]:
#speed_index
horse_results_baselog = (
    population.merge(
        horse_results, on="horse_id", suffixes=("", "_horse")
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
#基準タイムの選定
"""

（	基準
タイム	－	走破
タイム	）×	距離
指数	＋（	斥量	－	５５	）×	２	＋	馬場
指数	＋	８０	＝	スピード指数


１０００万クラスのタイムを基準
・3歳上 / 4歳以上
・1勝クラス
・良 / 稍重
・入線順位1～3着馬
上記条件の走破タイムの平均を出す
(牝馬限定戦を除く)
・3歳上 / 4歳以上
・2勝クラス
・良 / 稍重
・入線順位1～3着馬
上記条件の走破タイムの平均を出す
(牝馬限定戦を除く)
それらを更に平均する

データ数が少ないコースでは、存在するクラスの平均走破タイムを
「クラス指数の指数差 ÷ 距離指数 ÷ 10」で秒換算し、基準タイムを補正する。
"""
baselog = (
    population.merge(
        race_info[["race_id", "course_len", "race_type","race_grade","ground_state", "weather","place"]], on="race_id"
    )
)

df = (
    baselog
    .merge(results[["race_id", "horse_id", "wakuban", "nobori","time","umaban","rank"]], on=["race_id", "horse_id"])
)
df["nobori"] = df["nobori"].fillna(df["nobori"].mean())

df["place"] = df["place"].astype(int)
df["race_grade"] = df["race_grade"].astype(int)
df["ground_state"] = df["ground_state"].astype(int)
df["weather"] = df["weather"].astype(int)  
df["distance_place_type"] = (df["course_len"].astype(str) + df["place"].astype(str) + df["race_type"].astype(str)).astype(int)
df2 = df
# 1. 計算したいrace_gradeのリスト
grades = [55,60,70,79,85,  89,  91,  94,  98]

# 2. 各race_gradeごとにtimeとnoboriの平均を計算し、結合
for grade in grades:
    # race_gradeが指定された値のときのtimeとnoboriの平均を計算
    time_nobori_avg = (
        df[
        (df['race_grade'] == grade) &  # race_gradeが指定のgrade
        (df['ground_state'].isin([0, 2])) &  # ground_stateが0または2
        (df['rank'].isin([1, 2, 3]))  # rankが1, 2, 3
        ]
        .groupby(['course_len', 'place', 'race_type'])[['time', 'nobori']]  # 3つのカテゴリごとにtimeとnoboriを集計
        .mean()
        .reset_index()  # インデックスをリセット
        .rename(columns={'time': f'time_avg_{grade}', 'nobori': f'nobori_avg_{grade}'})  # 列名を変更
    )
    
    # 元のDataFrameにマージ（left join）
    horse_results_baselog = pd.merge(horse_results_baselog, time_nobori_avg, on=['course_len', 'place', 'race_type'], how='left')
# 1. 補完する順番を指定
grades = [70, 79,85, 89, 60, 91,  94, 55, 98]

# 2. まずhorse_results_baselog内の補完処理を行う
horse_results_baselog['final_time_avg'] = np.nan
horse_results_baselog['final_nobori_avg'] = np.nan

# 3. 最初に85で補完
horse_results_baselog['final_time_avg'] = np.where(
    horse_results_baselog['final_time_avg'].isna(), 
    horse_results_baselog['time_avg_70'], 
    horse_results_baselog['final_time_avg']
)
horse_results_baselog['final_nobori_avg'] = np.where(
    horse_results_baselog['final_nobori_avg'].isna(), 
    horse_results_baselog['nobori_avg_70'], 
    horse_results_baselog['final_nobori_avg']
)


"""
距離指数をかける
距離	芝	ダート
1000m	1.8	1.7
1150m	1.52	1.45
1200m	1.45	1.39
1300m	1.34	1.27
1400m	1.23	1.18
1500m	1.12	1.08
1600m	1.06	1.02
1700m	1.00	0.94
1800m	0.93	0.88
1900m	0.88	0.83
2000m	0.83	0.79
2100m	0.79	0.75
2200m	0.75	0.7
2300m	0.71	0.67
2400m	0.68	0.64
2500m	0.64	0.61
2600m	0.62	0.59
2800m	0.56	0.53
3000m	0.53	0.5
3200m	0.50	0.47
3400m	0.47	0.44
3600m	0.45	0.42
"""
# 距離と芝ダートの対応表を辞書として定義
conversion_table = {
    1000: {1: 1.8, 0: 1.7},
    1150: {1: 1.52, 0: 1.45},
    1200: {1: 1.45, 0: 1.39},
    1300: {1: 1.34, 0: 1.27},
    1400: {1: 1.23, 0: 1.18},
    1500: {1: 1.12, 0: 1.08},
    1600: {1: 1.06, 0: 1.02},
    1700: {1: 1.00, 0: 0.94},
    1800: {1: 0.93, 0: 0.88},
    1900: {1: 0.88, 0: 0.83},
    2000: {1: 0.83, 0: 0.79},
    2100: {1: 0.79, 0: 0.75},
    2200: {1: 0.75, 0: 0.7},
    2300: {1: 0.71, 0: 0.67},
    2400: {1: 0.68, 0: 0.64},
    2500: {1: 0.64, 0: 0.61},
    2600: {1: 0.62, 0: 0.59},
    2800: {1: 0.56, 0: 0.53},
    3000: {1: 0.53, 0: 0.5},
    3200: {1: 0.5, 0: 0.47},
    3400: {1: 0.47, 0: 0.44},
    3600: {1: 0.45, 0: 0.42}
}

# 新しい列に変換後の値を格納
def map_conversion(row):
    course_len = row['course_len']
    race_type = row['race_type']
    if course_len in conversion_table and race_type in conversion_table[course_len]:
        return conversion_table[course_len][race_type]
    else:
        return None  # 該当しない場合はNoneを返す

# 適用
horse_results_baselog['converted_value'] = horse_results_baselog.apply(map_conversion, axis=1)


# 4. 残りのグレードで補完（順番に）
for grade in grades:
    if grade != 70:
        time_col = f'time_avg_{grade}'
        nobori_col = f'nobori_avg_{grade}'
        
        # timeの欠損を補完
        horse_results_baselog['final_time_avg'] = np.where(
            horse_results_baselog['final_time_avg'].isna(), 
            horse_results_baselog[time_col] +  ((((grade - 70) / 1.2)/horse_results_baselog['converted_value'])/10),
            horse_results_baselog['final_time_avg']
        )
        # noboriの欠損を補完
        horse_results_baselog['final_nobori_avg'] = np.where(
            horse_results_baselog['final_nobori_avg'].isna(), 
            horse_results_baselog[nobori_col], 
            horse_results_baselog['final_nobori_avg']
        )



# 1. final_time_avgからtimeを引いた数値（秒）を計算
horse_results_baselog['time_diff_sp'] = horse_results_baselog['final_time_avg'] - horse_results_baselog['time']

# 2. time_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
horse_results_baselog['time_points'] = (horse_results_baselog['time_diff_sp'])

# 1. final_nobori_avgからnoboriを引いた数値（秒）を計算
horse_results_baselog['nobori_diff_sp'] = horse_results_baselog['final_nobori_avg'] - horse_results_baselog['nobori']

# 2. nobori_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
horse_results_baselog['nobori_points'] = (horse_results_baselog['nobori_diff_sp'])



# 2. time_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
horse_results_baselog['time_points_course_index'] = horse_results_baselog['time_points'] *horse_results_baselog['converted_value'] 

# 2. nobori_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
horse_results_baselog['nobori_points_course_index'] = horse_results_baselog['nobori_points'] *horse_results_baselog['converted_value'] 


"""
＋（	斥量	－	５５	）
"""
# 2. time_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
horse_results_baselog['time_points_impost'] = (horse_results_baselog['time_points_course_index'] +(horse_results_baselog["impost"]-55) *1.7)

# 2. nobori_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
horse_results_baselog['nobori_points_impost'] = (horse_results_baselog['nobori_points_course_index']+(horse_results_baselog["impost"]-55) *1.7)



"""
暫定馬場指数＝（馬場指数用基準タイム－該当レース上位３頭の平均タイム）× 距離指数
馬場指数用基準タイム ＝ 基準タイム － (クラス指数 × 距離指数)　＋pase_diff

ハイペースなら低く（早く見える）でてしまう
スローペースなら高く（遅く見える）でてしまう
pase_diffは+だとハイペース
-だとスローペースなので
そのまま+してよいかも
"""

# 2. 各race_gradeごとにtimeとnoboriの平均を計算し、結合
for grade in grades:
    # race_gradeが指定された値のときのtimeとnoboriの平均を計算
    time_nobori_avg_top = (
        df2[
        (df2['race_grade'] == grade) &  # race_gradeが指定のgrade
        (df2['ground_state'].isin([0, 2])) &  # ground_stateが0または2
        (df2['rank'].isin([1,2,3]))  # rankが1, 2, 3
        ]
        .groupby(['course_len', 'place', 'race_type'])[['time', 'nobori']]  # 3つのカテゴリごとにtimeとnoboriを集計
        .mean()
        .reset_index()  # インデックスをリセット
        .rename(columns={'time': f'time_avg_{grade}_top', 'nobori': f'nobori_avg_{grade}_top'})  # 列名を変更
    )
    
    # 元のDataFrameにマージ（left join）
    horse_results_baselog = pd.merge(horse_results_baselog, time_nobori_avg_top, on=['course_len', 'place', 'race_type'], how='left')
# 1. 補完する順番を指定
grades = [70, 79,85, 89, 60, 91,  94, 55, 98]

# 2. まずhorse_results_baselog内の補完処理を行う
horse_results_baselog['final_time_avg_top'] = np.nan
horse_results_baselog['final_nobori_avg_top'] = np.nan

# 3. 最初に85で補完
horse_results_baselog['final_time_avg_top'] = np.where(
    horse_results_baselog['final_time_avg_top'].isna(), 
    horse_results_baselog['time_avg_70_top'], 
    horse_results_baselog['final_time_avg_top']
)
horse_results_baselog['final_nobori_avg_top'] = np.where(
    horse_results_baselog['final_nobori_avg_top'].isna(), 
    horse_results_baselog['nobori_avg_70_top'], 
    horse_results_baselog['final_nobori_avg_top']
)

# 4. 残りのグレードで補完（順番に）
for grade in grades:
    if grade != 70:
        time_col = f'time_avg_{grade}_top'
        nobori_col = f'nobori_avg_{grade}_top'
        
        # timeの欠損を補完
        horse_results_baselog['final_time_avg_top'] = np.where(
            horse_results_baselog['final_time_avg_top'].isna(), 
            horse_results_baselog[time_col] + ((((grade - 70) / 1.2)/horse_results_baselog['converted_value'])/10),
            horse_results_baselog['final_time_avg_top']
        )
        # noboriの欠損を補完
        horse_results_baselog['final_nobori_avg_top'] = np.where(
            horse_results_baselog['final_nobori_avg_top'].isna(), 
            horse_results_baselog[nobori_col], 
            horse_results_baselog['final_nobori_avg_top']
        )



horse_results_baselog['time_condition_index_shaft'] = horse_results_baselog['final_time_avg_top']-(((horse_results_baselog['race_grade'] - 70)/ 12)*horse_results_baselog['converted_value'])
horse_results_baselog['nobori_condition_index_shaft'] = horse_results_baselog['final_nobori_avg_top']

# rankが1の場合はそのまま、rankが2以上の場合はtimeからrank_diffを引く
horse_results_baselog['adjusted_time'] = horse_results_baselog.apply(
    lambda row: row['time'] + 0.15 if row['rank'] == 1 else row['time'] - row['rank_diff']+1+ 0.15,
    axis=1
)


# 1. final_time_avgからtimeを引いた数値（秒）を計算
horse_results_baselog['time_diff_grade'] = horse_results_baselog['time_condition_index_shaft'] - horse_results_baselog['adjusted_time']

# 2. time_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
horse_results_baselog['time_points_grade'] = (horse_results_baselog['time_diff_grade'] )

# 1. final_nobori_avgからnoboriを引いた数値（秒）を計算
horse_results_baselog['nobori_diff_grade'] = horse_results_baselog['nobori_condition_index_shaft']

# 2. nobori_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
horse_results_baselog['nobori_points_grade'] = (horse_results_baselog['time_diff_grade'] )

"""
距離指数をかける
"""

# 2. time_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
horse_results_baselog['time_points_grade_index'] = (horse_results_baselog['time_points_grade'] *horse_results_baselog['converted_value'])

# 2. nobori_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
horse_results_baselog['nobori_points_grade_index'] = (horse_results_baselog['nobori_points_grade'] *horse_results_baselog['converted_value'])

horse_results_baselog['time_condition_index'] = horse_results_baselog['time_points_grade_index'] +horse_results_baselog['pace_diff'] *1/3
horse_results_baselog['nobori_condition_index'] = horse_results_baselog['nobori_points_grade_index'] +horse_results_baselog['pace_diff'] *1/3


# 新しい列を作成
horse_results_baselog['race_grade_transformed'] = (horse_results_baselog['race_grade'] - 80) / 5 + 80


horse_results_baselog['speed_index'] = horse_results_baselog['time_points_impost'] + horse_results_baselog['time_condition_index'] + horse_results_baselog['race_grade_transformed'] 
horse_results_baselog['nobori_index'] = horse_results_baselog['nobori_points_impost'] + horse_results_baselog['nobori_condition_index'] + horse_results_baselog['race_grade_transformed'] 


# 5. 不要な中間列を削除し、必要な列だけ残す
columns_to_drop = [f'time_avg_{grade}' for grade in grades] + [f'nobori_avg_{grade}' for grade in grades]
horse_results_baselog = horse_results_baselog.drop(columns=columns_to_drop)



# horse_results_baselog = horse_results_baselog[['race_id',
# 'date',
# 'horse_id',
# 'time_diff_sp',
# 'nobori_diff_sp',
# 'time_points_course_index',
# 'nobori_points_course_index',
# 'time_points_impost',
# 'nobori_points_impost',
# 'time_diff_grade',
# 'nobori_diff_grade',
# 'time_points_grade_index',
# 'nobori_points_grade_index',
# 'time_condition_index',
# 'nobori_condition_index',
# 'speed_index',
# 'nobori_index']]

# grouped_df = horse_results_baselog.groupby(["race_id", "horse_id"])
# merged_df = self.population.copy()
# for n_race in tqdm(n_races, desc=f"speed_index"):
#     df_speed = (
#         grouped_df.head(n_race)
#         .groupby(["race_id", "horse_id"])[
#             [
#                 'time_diff_sp',
#                 'nobori_diff_sp',
#                 'time_points_course_index',
#                 'nobori_points_course_index',
#                 'time_points_impost',
#                 'nobori_points_impost',
#                 'time_diff_grade',
#                 'nobori_diff_grade',
#                 'time_points_grade_index',
#                 'nobori_points_grade_index',
#                 'time_condition_index',
#                 'nobori_condition_index',
#                 'speed_index',
#                 'nobori_index',
#             ]
#         ]
#         .agg(["mean", "max", "min"])
#     )
#     original_df = df_speed.copy()
#     df_speed.columns = [
#         "_".join(col) + f"_{n_race}races" for col in df_speed.columns
#     ]
#     # レースごとの相対値に変換
#     tmp_df = df_speed.groupby(["race_id"])
#     relative_df = ((df_speed - tmp_df.mean()) / tmp_df.std()).add_suffix("_relative")
#     merged_df = merged_df.merge(
#         relative_df, on=["race_id", "horse_id"], how="left"
#     )
#     # 相対値を付けない元の列をそのまま追加
#     original_df.columns = [
#         "_".join(col) + f"_{n_race}races" for col in original_df.columns
#     ]  # 列名変更
    
#     merged_df = merged_df.merge(
#         original_df, on=["race_id", "horse_id"], how="left"
#     )


# self.agg_cross_features_df_15 = merged_df

In [113]:
horse_results_baselog_1 = horse_results_baselog[['race_id',
'race_class',
'race_grade_transformed',
'date',
'horse_id','speed_index',
'nobori_index']]

In [114]:
horse_results_baselog_1.head(188)


,race_id,race_class,race_grade_transformed,date,horse_id,speed_index,nobori_index
0,202306050712,5,81.8,2023-12-23,2018105873,69.842431,69.355875
1,202309050907,2,78.0,2023-12-28,2016100742,85.036858,83.794557
2,202309050910,5,81.8,2023-12-28,2020103061,83.158687,82.136140
3,202309050712,2,78.0,2023-12-23,2020103664,78.261595,79.131071
4,202309050812,2,78.0,2023-12-24,2020105154,82.974179,81.707615
5,202306050907,2,78.0,2023-12-28,2019100777,83.105415,80.925423
6,202309050907,2,78.0,2023-12-28,2019100740,77.424415,75.493423
7,202306050912,3,79.8,2023-12-28,2019107075,88.121233,87.045687
8,202309050912,4,81.0,2023-12-28,2017106240,84.616597,84.420042
9,202309050910,2,78.0,2023-12-28,2020103696,82.567415,80.470423


In [115]:

# race_classが8の行だけを表示
horse_results_baselog_1[horse_results_baselog_1['race_class'] == 8].head(188)


,race_id,race_class,race_grade_transformed,date,horse_id,speed_index,nobori_index
100,202306050811,8,83.6,2023-12-24,2018103559,91.752578,89.621156
102,202306050811,8,83.6,2023-12-24,2017102170,91.412578,89.825156
104,202306050811,8,83.6,2023-12-24,2019104740,88.692578,86.901156
106,202306050811,8,83.6,2023-12-24,2019105283,92.024578,90.505156
107,202306050711,8,83.6,2023-12-23,2017101436,89.644578,89.553156
147,202309050711,8,83.6,2023-12-23,2019103034,88.591752,86.972842
150,202309050711,8,83.6,2023-12-23,2020105295,87.209752,85.590842
171,202309050311,8,83.6,2023-12-09,2019101348,88.803752,87.290842
300,202309050111,8,83.6,2023-12-02,2017102294,86.619697,85.253788
313,202306050811,8,83.6,2023-12-24,2019103588,86.769697,85.403788


In [62]:

#基準タイムの選定
"""

（	基準
タイム	－	走破
タイム	）×	距離
指数	＋（	斥量	－	５５	）×	２	＋	馬場
指数	＋	８０	＝	スピード指数


１０００万クラスのタイムを基準
・3歳上 / 4歳以上
・1勝クラス
・良 / 稍重
・入線順位1～3着馬
上記条件の走破タイムの平均を出す
(牝馬限定戦を除く)
・3歳上 / 4歳以上
・2勝クラス
・良 / 稍重
・入線順位1～3着馬
上記条件の走破タイムの平均を出す
(牝馬限定戦を除く)
それらを更に平均する

データ数が少ないコースでは、存在するクラスの平均走破タイムを
「クラス指数の指数差 ÷ 距離指数 ÷ 10」で秒換算し、基準タイムを補正する。
"""
baselog = (
    population.merge(
        race_info[["race_id", "course_len", "race_class","race_type","race_grade","ground_state", "weather","place","race_date_day_count"]], on="race_id"
    )
)

df = (
    baselog
    .merge(results[["race_id", "impost","horse_id", "wakuban", "nobori","time","umaban","rank"]], on=["race_id", "horse_id"])
)
df["nobori"] = df["nobori"].fillna(df["nobori"].mean())

df["place"] = df["place"].astype(int)
df["race_grade"] = df["race_grade"].astype(int)
df["ground_state"] = df["ground_state"].astype(int)
df["weather"] = df["weather"].astype(int)  
df["distance_place_type"] = (df["course_len"].astype(str) + df["place"].astype(str) + df["race_type"].astype(str)).astype(int)
df2 = df
# 1. 計算したいrace_gradeのリスト
grades = [55,60,70,79,85,  89,  91,  94,  98]

# 2. 各race_gradeごとにtimeとnoboriの平均を計算し、結合
for grade in grades:
    # race_gradeが指定された値のときのtimeとnoboriの平均を計算
    time_nobori_avg = (
        df[
        (df['race_grade'] == grade) &  # race_gradeが指定のgrade
        (df['ground_state'].isin([0, 2])) &  # ground_stateが0または2
        (df['rank'].isin([1, 2, 3]))  # rankが1, 2, 3
        ]
        .groupby(['course_len', 'place', 'race_type'])[['time', 'nobori']]  # 3つのカテゴリごとにtimeとnoboriを集計
        .mean()
        .reset_index()  # インデックスをリセット
        .rename(columns={'time': f'time_avg_{grade}', 'nobori': f'nobori_avg_{grade}'})  # 列名を変更
    )
    
    # 元のDataFrameにマージ（left join）
    df = pd.merge(df, time_nobori_avg, on=['course_len', 'place', 'race_type'], how='left')
# 1. 補完する順番を指定
grades = [70, 79,85, 89, 60, 91,  94, 55, 98]

# 2. まずhorse_results_baselog内の補完処理を行う
df['final_time_avg'] = np.nan
df['final_nobori_avg'] = np.nan

# 3. 最初に85で補完
df['final_time_avg'] = np.where(
    df['final_time_avg'].isna(), 
    df['time_avg_70'], 
    df['final_time_avg']
)
df['final_nobori_avg'] = np.where(
    df['final_nobori_avg'].isna(), 
    df['nobori_avg_70'], 
    df['final_nobori_avg']
)


"""
距離指数をかける
距離	芝	ダート
1000m	1.8	1.7
1150m	1.52	1.45
1200m	1.45	1.39
1300m	1.34	1.27
1400m	1.23	1.18
1500m	1.12	1.08
1600m	1.06	1.02
1700m	1.00	0.94
1800m	0.93	0.88
1900m	0.88	0.83
2000m	0.83	0.79
2100m	0.79	0.75
2200m	0.75	0.7
2300m	0.71	0.67
2400m	0.68	0.64
2500m	0.64	0.61
2600m	0.62	0.59
2800m	0.56	0.53
3000m	0.53	0.5
3200m	0.50	0.47
3400m	0.47	0.44
3600m	0.45	0.42
"""
# 距離と芝ダートの対応表を辞書として定義
conversion_table = {
    1000: {1: 1.8, 0: 1.7},
    1150: {1: 1.52, 0: 1.45},
    1200: {1: 1.45, 0: 1.39},
    1300: {1: 1.34, 0: 1.27},
    1400: {1: 1.23, 0: 1.18},
    1500: {1: 1.12, 0: 1.08},
    1600: {1: 1.06, 0: 1.02},
    1700: {1: 1.00, 0: 0.94},
    1800: {1: 0.93, 0: 0.88},
    1900: {1: 0.88, 0: 0.83},
    2000: {1: 0.83, 0: 0.79},
    2100: {1: 0.79, 0: 0.75},
    2200: {1: 0.75, 0: 0.7},
    2300: {1: 0.71, 0: 0.67},
    2400: {1: 0.68, 0: 0.64},
    2500: {1: 0.64, 0: 0.61},
    2600: {1: 0.62, 0: 0.59},
    2800: {1: 0.56, 0: 0.53},
    3000: {1: 0.53, 0: 0.5},
    3200: {1: 0.5, 0: 0.47},
    3400: {1: 0.47, 0: 0.44},
    3600: {1: 0.45, 0: 0.42}
}

# 新しい列に変換後の値を格納
def map_conversion(row):
    course_len = row['course_len']
    race_type = row['race_type']
    if course_len in conversion_table and race_type in conversion_table[course_len]:
        return conversion_table[course_len][race_type]
    else:
        return None  # 該当しない場合はNoneを返す

# 適用
df['converted_value'] = df.apply(map_conversion, axis=1)


# 4. 残りのグレードで補完（順番に）
for grade in grades:
    if grade != 70:
        time_col = f'time_avg_{grade}'
        nobori_col = f'nobori_avg_{grade}'
        
        # timeの欠損を補完
        df['final_time_avg'] = np.where(
            df['final_time_avg'].isna(), 
            df[time_col] + ((grade - 70) / 1.2/df['converted_value']/10),
            df['final_time_avg']
        )
        # noboriの欠損を補完
        df['final_nobori_avg'] = np.where(
            df['final_nobori_avg'].isna(), 
            df[nobori_col], 
            df['final_nobori_avg']
        )



# 1. final_time_avgからtimeを引いた数値（秒）を計算
df['time_diff_sp'] = df['final_time_avg'] - df['time']

# 2. time_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
df['time_points'] = (df['time_diff_sp'])

# 1. final_nobori_avgからnoboriを引いた数値（秒）を計算
df['nobori_diff_sp'] = df['final_nobori_avg'] - df['nobori']

# 2. nobori_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
df['nobori_points'] = (df['nobori_diff_sp'])



# 2. time_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
df['time_points_course_index'] = df['time_points'] *df['converted_value'] 

# 2. nobori_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
df['nobori_points_course_index'] = df['nobori_points'] *df['converted_value'] 


"""
＋（	斥量	－	５５	）
"""
# 2. time_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
df['time_points_impost'] = (df['time_points_course_index'] +(df["impost"]-55) *1.7)

# 2. nobori_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
df['nobori_points_impost'] = (df['nobori_points_course_index']+(df["impost"]-55) *1.7)



"""
暫定馬場指数＝（馬場指数用基準タイム－該当レース上位３頭の平均タイム）× 距離指数
馬場指数用基準タイム ＝ 基準タイム － (クラス指数 × 距離指数)　＋pase_diff

ハイペースなら低く（早く見える）でてしまう
スローペースなら高く（遅く見える）でてしまう
pase_diffは+だとハイペース
-だとスローペースなので
そのまま+してよいかも
"""
#predictの時にはold_poplationに変える
old_merged_df = population.copy()     
#ここで、直近レースのtimeを知りたい
df_old2 = (
    old_merged_df
    .merge(results[["race_id", "horse_id","time","rank","rank_per_horse"]], on=["race_id", "horse_id"])
    .merge(race_info[["race_id", "place","weather","ground_state","race_grade","course_len","race_type","race_date_day_count"]], on="race_id")
)
df_old2["place"] = df_old2["place"].astype(int)
df_old2["race_grade"] = df_old2["race_grade"].astype(int)
# 距離/競馬場/タイプ/レースランク
df_old2["distance_place_type_race_grade"] = (df_old2["course_len"].astype(str) + df_old2["place"].astype(str) + df_old2["race_type"].astype(str) + df_old2["race_grade"].astype(str)).astype(int)


baselog_old = (
    population.merge(
        race_info[["race_id", "course_len", "race_type","race_grade","ground_state", "weather","place","around"]], on="race_id"
    )
    # .merge(
    #     self.horse_results,
    #     on=["horse_id", "course_len", "race_type"],
    #     suffixes=("", "_horse"),
    # )
    # .query("date_horse < date")
    # .sort_values("date_horse", ascending=False)
)
df_old = (
    baselog_old
    .merge(results[["race_id", "horse_id", "wakuban", "umaban","nobori","rank","time","sex"]], on=["race_id", "horse_id"])
)
df_old["place"] = df_old["place"].astype(int)
df_old["race_grade"] = df_old["race_grade"].astype(int)
df_old["ground_state"] = df_old["ground_state"].astype(int)
df_old["around"] = df_old["around"].fillna(3).astype(int)
df_old["weather"] = df_old["weather"].astype(int)  
             

df_old["distance_place_type_race_grade"] = (df_old["course_len"].astype(str) + df_old["place"].astype(str) + df_old["race_type"].astype(str) + df_old["race_grade"].astype(str)).astype(int)
df_old_copy = df_old
# rank列が1, 2, 3の行だけを抽出
df_old = df_old[df_old['rank'].isin([1, 2, 3])]
target_mean_1 = df_old.groupby("distance_place_type_race_grade")["time"].mean()
df_old = df_old.copy()
# 平均値をカテゴリ変数にマッピング
df_old["distance_place_type_race_grade_encoded"] = df_old["distance_place_type_race_grade"].map(target_mean_1)


df_old = df_old[["distance_place_type_race_grade",'distance_place_type_race_grade_encoded']]


columns_to_merge = [
    ("distance_place_type_race_grade",'distance_place_type_race_grade_encoded'),
]

# 各ペアを順番に処理
for original_col, encoded_col in columns_to_merge:
    df2_subset = df_old[[original_col, encoded_col]].drop_duplicates()  # 重複を削除
    df2_subset = df2_subset.reset_index(drop=True)  # インデックスをリセット
    df_old2 = df_old2.merge(df2_subset, on=original_col, how='left')  # dfにマージ
df_old2 = df_old2[df_old2['rank'].isin([1, 2, 3])]
df_old2["distance_place_type_race_grade_encoded_time_diff"] = df_old2['distance_place_type_race_grade_encoded'] - df_old2["time"]

# df_old2= df_old2[df_old2["race_type"] != 2]
# df_old2_1 = df_old2[df_old2["race_type"] != 0]
df_old2_1 = df_old2
# 2. df の各行について処理
def compute_mean_for_row(row, df_old2_1):
    # race_type == 0 の場合は NaN を返す
    # if row["race_type"] == 0:
    #     return np.nan
        
    target_day_count = row["race_date_day_count"]  # df の各行の race_date_day_count

    # 3. df_old2_1 から条件に一致する行をフィルタリング
    filtered_df_old2_1 = df_old2_1[
        (df_old2_1["race_date_day_count"] >= (target_day_count - 1200)) &  # race_date_day_count が target_day_count-1200 以上
        (df_old2_1["race_date_day_count"] <= (target_day_count - 1)) &  # race_date_day_count が target_day_count-1 以下
        (df_old2_1["place"] == row["place"]) &  # place が一致
        (df_old2_1["race_type"] == row["race_type"])  & 
        (df_old2_1["weather"].isin([0, 1, 2])) &  # weather が 0, 1, 2 のいずれか
        (df_old2_1["ground_state"] == 0)   # ground_state が 0
        # &
        # (df_old2_1["rank_per_horse"] < 0.87)  # rank_per_horse が 0.87 未満
    ]

    # 4. フィルタリングした行の "distance_place_type_race_grade_encoded_time_diff" の平均を計算
    mean_time_diff = filtered_df_old2_1["distance_place_type_race_grade_encoded_time_diff"].mean()

    # 5. 計算結果を返す（NaNの場合も考慮）
    return mean_time_diff if not np.isnan(mean_time_diff) else np.nan

# 6. df の各行に対して、計算した平均値を新しい列に追加
df['time_diff_grade'] = df.apply(compute_mean_for_row, axis=1, df_old2_1=df_old2_1)


# 2. time_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
df['time_points_grade'] = (df['time_diff_grade'] )

# # 1. final_nobori_avgからnoboriを引いた数値（秒）を計算
# df['nobori_diff_grade'] = df['nobori_condition_index_shaft'] 

# 2. nobori_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
df['nobori_points_grade'] = (df['time_diff_grade'] )

"""
距離指数をかける
"""

# 2. time_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
df['time_points_grade_index'] = (df['time_points_grade'] *df['converted_value'])

# 2. nobori_diffを0.1秒ごとのポイントに変換（1ポイント = 0.1秒）
df['nobori_points_grade_index'] = (df['nobori_points_grade'] *df['converted_value'])

df['time_condition_index'] = df['time_points_grade_index']
df['nobori_condition_index'] = df['nobori_points_grade_index']



# 新しい列を作成
horse_results_baselog['race_grade_transformed'] = (horse_results_baselog['race_grade'] - 80) / 5 + 80


df['speed_index'] = df['time_points_impost'] + df['time_condition_index'] +horse_results_baselog['race_grade_transformed']
df['nobori_index'] = df['nobori_points_impost'] + df['nobori_condition_index'] + horse_results_baselog['race_grade_transformed']


# 5. 不要な中間列を削除し、必要な列だけ残す
columns_to_drop = [f'time_avg_{grade}' for grade in grades] + [f'nobori_avg_{grade}' for grade in grades]
df = df.drop(columns=columns_to_drop)



df = df[['race_id',
         'race_class',
'date',
'horse_id','speed_index',
'nobori_index']]

merged_df = population.copy()
merged_df = merged_df.merge(df, on=["race_id",'date', "horse_id"], how="left")


In [107]:
merged_df.head(100)

# race_classが8の行だけを表示
merged_df[merged_df['race_class'] == 8].head(188)

,race_id,date,horse_id,race_class,speed_index,nobori_index
0,201906010105,2019-01-05,2016104656,0.0,NaN,NaN
1,201906010105,2019-01-05,2016103323,0.0,NaN,NaN
2,201906010105,2019-01-05,2016105106,0.0,NaN,NaN
3,201906010105,2019-01-05,2016102787,0.0,NaN,NaN
4,201906010105,2019-01-05,2016102128,0.0,NaN,NaN
5,201906010105,2019-01-05,2016106379,0.0,NaN,NaN
6,201906010105,2019-01-05,2016103769,0.0,NaN,NaN
7,201906010105,2019-01-05,2016105261,0.0,NaN,NaN
8,201906010105,2019-01-05,2016105293,0.0,NaN,NaN
9,201906010105,2019-01-05,2016103312,0.0,NaN,NaN
